In [10]:
from itertools import product

from tqdm.notebook import tqdm

In [11]:
import highspy


def compile_problem(n, m, include_total=True):
    h = highspy.Highs()
    # h.setOptionValue("time_limit", 0.1)
    h.setOptionValue("presolve", "off")
    # h.setOptionValue("simplex_iteration_limit", 64)
    h.setOptionValue("log_to_console", True)

    vs = {(i, j): h.addVariable(lb=0.0, ub=1.0) for i, j in product(range(n), range(m))}
    for i in range(n):
        h.addConstr(sum(vs[i, j] for j in range(m)) == 1.0)
    for j in range(m):
        h.addConstr(sum(vs[i, j] for i in range(n)) <= 1.0)
    if include_total:
        h.addConstr(sum(vs[i, j] for i in range(n) for j in range(m)) == 1.0)
    h.minimize(sum(vs[i, j] for i in range(n) for j in range(m)))

    return h

In [12]:
n_values = list(range(8, 101, 8))
m_values = list(range(8, 101, 8))
count = len(n_values) * len(m_values)
args_values = list(product(n_values, m_values))

In [13]:
# for args in tqdm(args_values, total=count):
for args in tqdm(args_values, total=count):
    problem = compile_problem(*args)
    key = "_".join(map(str, args))
    output_path = f"../resources/highspy/{key}.mps"
    result = problem.writeModel(output_path)

  0%|          | 0/144 [00:00<?, ?it/s]

In [14]:
h = highspy.Highs()

In [19]:
%%timeit
compile_problem(64, 64)

83.3 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
h.passModel(h.getLp())

75.6 μs ± 352 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%%timeit
problem = h.readModel("../resources/highspy/64_64.mps")

10.7 ms ± 167 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
